In [1]:
import pandas as pd
import numpy as np
import csv

In [10]:
# https://theleftjoin.com/how-to-write-a-pandas-dataframe-to-an-sqlite-table/

In [2]:
# import csv
# df.id = df.id.astype(str)
# df.to_csv('test.txt', index=False, quotechar='"',header=None, quoting=csv.QUOTE_NONNUMERIC)

### ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [3]:
dep_df_temp = pd.read_excel('datos_diana/Indicadores2021_departamentos.xlsx')
mun_df_temp = pd.read_excel('datos_diana/Indicadores2021_municipios.xlsx')

## 01_Departamentos.csv

In [4]:
dep_df = dep_df_temp.copy()
dep_df = dep_df[['DIVIPOLA', 'DEPARTAMENTO']]
dep_df['DIVIPOLA'] = dep_df['DIVIPOLA'].astype(str).str.zfill(2)
dep_df['DEPARTAMENTO'] = dep_df['DEPARTAMENTO'].str.strip().str.capitalize()
dep_df = dep_df.drop_duplicates();

dep_df.reset_index(inplace=True)
dep_df['index'] = dep_df['index'] + 1

dep_df = dep_df[['index', 'DIVIPOLA', 'DEPARTAMENTO']]
dep_df.rename(columns = {'index':'id', 'DIVIPOLA':'divipola', 'DEPARTAMENTO':'nombre'}, inplace = True)

# dep_df.to_excel('./01_Departamentos.xlsx', index=False)
dep_df.to_csv('./01_Departamentos.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
dep_df.head()

,id,divipola,nombre
0,1,05,Antioquia
1,2,08,Atlántico
2,3,11,Bogotá d.c
3,4,13,Bolívar
4,5,15,Boyacá


## 02_Municipios.csv

In [5]:
mun_df = mun_df_temp.copy()
mun_df = mun_df[['DIVIPOLA', 'MUNICIPIO']]
mun_df['DIVIPOLA'] = mun_df['DIVIPOLA'].astype(str).str.zfill(5)
mun_df['MUNICIPIO'] = mun_df['MUNICIPIO'].str.strip().str.capitalize()
mun_df = mun_df.drop_duplicates();

mun_df["dep_divipola"] = mun_df['DIVIPOLA'].str[:2]
mun_df = mun_df.merge(dep_df[['id', 'divipola']], left_on='dep_divipola', right_on='divipola', how='left')

mun_df.reset_index(inplace=True)
mun_df['index'] = mun_df['index'] + 1

mun_df = mun_df[['index', 'DIVIPOLA', 'MUNICIPIO', 'id',]]
mun_df.rename(columns = {'index':'id', 'id':'departamento_id', 'DIVIPOLA':'divipola', 'MUNICIPIO':'nombre'}, inplace = True)
mun_df['departamento'] = ''

# mun_df.to_excel('./02_Municipios.xlsx', index=False)
mun_df.to_csv('./02_Municipios.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
mun_df.head()

,id,divipola,nombre,departamento_id,departamento
0,1,05001,Medellín,1,
1,2,05002,Abejorral,1,
2,3,05004,Abriaquí,1,
3,4,05021,Alejandría,1,
4,5,05030,Amagá,1,


## 03_Dimensiones.csv

In [6]:
dim_df = pd.concat([dep_df_temp['Dimensión'], mun_df_temp['Dimensión']]).to_frame()
dim_df.rename(columns = {'Dimensión':'nombre'}, inplace = True)
dim_df['nombre'] = dim_df['nombre'].replace('Ind_Pobreza_Multidimensional','Indice de Pobreza Multidimensional y tasas mortalidad')
dim_df['nombre'] = dim_df['nombre'].str.strip().str.capitalize()
dim_df = dim_df.drop_duplicates();
dim_df.sort_values(by=['nombre'], inplace=True)

dim_df.reset_index(drop=True, inplace=True)
dim_df.reset_index(inplace=True)
dim_df['index'] = dim_df['index'] + 1

dim_df.rename(columns = {'index':'id'}, inplace = True)
# dim_df.to_excel('./03_Dimensiones.xlsx', index=False)
dim_df.to_csv('./03_Dimensiones.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
dim_df.head()

,id,nombre
0,1,Accesibilidad
1,2,Calidad
2,3,Disponibilidad
3,4,Indice de pobreza multidimensional y tasas mor...
4,5,Población


## 04_Indicadores.csv

In [7]:
def es_porcentual(indicador):
    if any(substring in indicador.lower() for substring in ['%', 'tasa']):
        return 1
    return 0

ind_df = pd.concat([dep_df_temp[['Dimensión', 'Nombre_tablero']], mun_df_temp[['Dimensión', 'Nombre_tablero']]])
ind_df.rename(columns = {'Dimensión':'dimension', 'Nombre_tablero':'indicador'}, inplace = True)
ind_df['dimension'] = ind_df['dimension'].replace('Ind_Pobreza_Multidimensional','Indice de Pobreza Multidimensional y tasas mortalidad')
ind_df['dimension'] = ind_df['dimension'].str.strip().str.capitalize()
ind_df['indicador'] = ind_df['indicador'].replace("% \'", "(%) ", regex=True)
ind_df['indicador'] = ind_df['indicador'].str.strip().str.capitalize()
ind_df = ind_df.drop_duplicates();
ind_df["es_porcentual"] = ind_df["indicador"].apply(es_porcentual)
ind_df.sort_values(by=['indicador'], inplace=True)

ind_df = ind_df.merge(dim_df[['nombre', 'id']], left_on='dimension', right_on='nombre', how='left')

ind_df.reset_index(drop=True, inplace=True)
ind_df.reset_index(inplace=True)
ind_df['index'] = ind_df['index'] + 1

ind_df = ind_df[['index', 'indicador', 'id', 'es_porcentual']]
ind_df.rename(columns = {'id':'dimension_id', 'index':'id', 'indicador':'nombre'}, inplace = True)
# ind_df.to_excel('./04_Indicadores.xlsx', index=False)
ind_df.to_csv('./04_Indicadores.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
ind_df.head()

,id,nombre,dimension_id,es_porcentual
0,1,(%) mujeres en edad reproductiva sin cobertura...,1,1
1,2,(%) mujeres en edad reproductiva sin cobertura...,1,1
2,3,(%) mujeres rurales sin cobertura en en edad r...,1,1
3,4,(%) mujeres urbanas sin cobertura en en edad r...,1,1
4,5,(%) población infantil rural sin cobertura,1,1


## 05_RegistroIndiceDepartamental.csv

In [8]:
# dep_df_temp

## 06_RegistroIndiceMunicipal.csv

In [9]:
reg_mun = mun_df_temp.copy()
reg_mun = reg_mun[['DIVIPOLA', 'Nombre_tablero', 'Valor']]

reg_mun['DIVIPOLA'] = reg_mun['DIVIPOLA'].astype(str).str.zfill(5)
reg_mun = reg_mun.merge(mun_df[['id', 'divipola']], left_on='DIVIPOLA', right_on='divipola', how='left')
reg_mun.drop(['DIVIPOLA', 'divipola'], axis=1, inplace=True)
reg_mun.rename(columns = {'id':'municipio_id'}, inplace = True)

reg_mun['Nombre_tablero'] = reg_mun['Nombre_tablero'].replace("% \'", "(%) ", regex=True)
reg_mun['Nombre_tablero'] = reg_mun['Nombre_tablero'].str.strip().str.capitalize()
reg_mun = reg_mun.merge(ind_df[['id', 'nombre']], left_on='Nombre_tablero', right_on='nombre', how='left')
reg_mun.drop(['nombre', 'Nombre_tablero'], axis=1, inplace=True)
reg_mun.rename(columns = {'id':'indicador_id'}, inplace = True)

reg_mun.reset_index(drop=True, inplace=True)
reg_mun.reset_index(inplace=True)
reg_mun['index'] = reg_mun['index'] + 1

reg_mun = reg_mun[['index', 'Valor', 'indicador_id', 'municipio_id']]
reg_mun.rename(columns = {'index':'id', 'Valor':'valor'}, inplace = True)
# reg_mun.to_excel('./06_RegistroIndiceMunicipal.xlsx', index=False)
# reg_mun.to_csv('./06_RegistroIndiceMunicipal.csv', index=False)
reg_mun.to_csv('./06_RegistroIndiceMunicipal.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
reg_mun.head()

,id,valor,indicador_id,municipio_id
0,1,12.8,33,1
1,2,40.6,33,2
2,3,34.8,33,3
3,4,31.9,33,4
4,5,22.9,33,5
